In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import os
import cv2
import tensorflow as tf
import gdown

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Input, Flatten, Dense, Reshape
from sklearn.preprocessing import LabelEncoder

In [2]:
url = 'https://drive.google.com/uc?id=1T-AQHx84kHVIVcH3wEcOmTEac7njLLlS'
output = 'train.zip'
gdown.download(url, output, quiet=True)

# https://drive.google.com/file/d/1T-AQHx84kHVIVcH3wEcOmTEac7njLLlS/view?usp=sharing

'train.zip'

In [3]:
url = 'https://drive.google.com/uc?id=15eg8RcRCbH8VnokjLU-UWEXwz8hREqGU'
output = 'test.zip'
gdown.download(url, output, quiet=True)

# https://drive.google.com/file/d/15eg8RcRCbH8VnokjLU-UWEXwz8hREqGU/view?usp=sharing

'test.zip'

In [4]:
DATA_DIR = '/content/'

In [5]:
train_zip = DATA_DIR + 'train.zip'
zip_ref = zipfile.ZipFile(train_zip, 'r')
zip_ref.extractall()
zip_ref.close()

test_zip = DATA_DIR + 'test.zip'
zip_ref = zipfile.ZipFile(test_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [6]:
print('Total training images:', len(os.listdir(DATA_DIR + 'Train')))
print('Total testing images:', len(os.listdir(DATA_DIR + 'Test')))

Total training images: 19906
Total testing images: 6636


In [7]:
train_df = pd.read_csv(DATA_DIR + 'train.csv')
test_df = pd.read_csv(DATA_DIR + 'test.csv')

In [8]:
train_df.head(1)

,ID,Class
0,377.jpg,MIDDLE


In [9]:
test_df.head(1)

,ID
0,25321.jpg


In [10]:
train_images = []
for img_name in train_df.ID:
    img_path = DATA_DIR + 'Train/' + img_name
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64)).astype('float32')
    train_images.append(img)

train_x = np.stack(train_images)
train_x = train_x / 255.

In [11]:
test_images = []
for img_name in test_df.ID:
    img_path = DATA_DIR + 'Test/' + img_name
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64, 64)).astype('float32')
    test_images.append(img)

test_x = np.stack(test_images)
test_x = test_x / 255.

In [12]:
lb = LabelEncoder()
train_y = lb.fit_transform(train_df.Class) # Class Names - > Class Numbers
train_y = tf.keras.utils.to_categorical(train_y) # Class Numbers -> OHE Vectors

In [31]:
from tensorflow.keras.applications.vgg19 import VGG19

base_model = VGG19(
    input_shape = (64, 64, 3),
    include_top = False,
    weights = 'imagenet'
    )

In [32]:
# for layer in base_model.layers:
#     layer.trainable = False

In [33]:
model= Sequential()
model.add(base_model) 
model.add(Flatten()) 
model.add(Dense(512, activation = ('relu')))
model.add(Dense(256, activation = ('relu')))
model.add(Dense(128, activation = ('relu')))
model.add(Dropout(.2))
model.add(Dense(3, activation = ('softmax')))

In [34]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                

In [36]:
model.fit(train_x,
          train_y,
          epochs = 50, 
          verbose = 1,
          validation_split = 0.2
          )

Epoch 1/50
498/498 [==============================] - 19s 36ms/step - loss: 0.8757 - accuracy: 0.5981 - val_loss: 0.6908 - val_accuracy: 0.7268
Epoch 2/50
498/498 [==============================] - 18s 35ms/step - loss: 0.6554 - accuracy: 0.7254 - val_loss: 0.5680 - val_accuracy: 0.7677
Epoch 3/50
498/498 [==============================] - 18s 35ms/step - loss: 0.5420 - accuracy: 0.7803 - val_loss: 0.4858 - val_accuracy: 0.7983
Epoch 4/50
498/498 [==============================] - 18s 35ms/step - loss: 0.4728 - accuracy: 0.8094 - val_loss: 0.4383 - val_accuracy: 0.8252
Epoch 5/50
498/498 [==============================] - 18s 35ms/step - loss: 0.3937 - accuracy: 0.8399 - val_loss: 0.4283 - val_accuracy: 0.8290
Epoch 6/50
498/498 [==============================] - 18s 35ms/step - loss: 0.3132 - accuracy: 0.8778 - val_loss: 0.3919 - val_accuracy: 0.8481
Epoch 7/50
498/498 [==============================] - 18s 35ms/step - loss: 0.2722 - accuracy: 0.8950 - val_loss: 0.4361 - val_accuracy:

In [37]:
def generate_prediction(model, test_x):
  pred = model.predict(test_x)
  pred = np.argmax(pred, axis = -1) # Returns the indices of the maximum values along an axis.
  pred = lb.inverse_transform(pred)
  return pred

In [38]:
test_df['Class'] = generate_prediction(model, test_x)
test_df.to_csv('prediction.csv', index = False)